## _*Using PySCF driver*_

Qiskit Chemistry supports a number of different chemistry drivers, i.e chemistry programs and software libraries, which are used to compute integrals that are then used to build the second quantized Hamiltonian in the FermionicOperator.

Drivers include Gaussian 16, PyQuante, PySCF, PSI4 and HDF5. The main Qiskit documentation has more information on [drivers](https://qiskit.org/documentation/aqua/chemistry/qiskit_chemistry_drivers.html).

For non-Windows platforms (where PySCF has no pre-built packages), the PySCF driver is installed as a  dependent when you `pip install qiskit-chemistry`. HDF5 support is built into Qiskit Chemistry. If you would like/prefer to use one of the other drivers then refer to the above link for installation and usage guidance.

Note: drivers were written to allow existing users of them to leverage creating the molecular input in a native way for the driver. While Multiplicity (2S+1) is commonly used to specify the overall spin of the molecule, PySCF uses Spin (2S) if you are programming directly with its API and that is what is exposed here. For a singlet system, as in the example below i.e. equal numbers of alpha and beta electrons, the overall spin here is 0 and 2S is 0 (Multiplicity would have been 1).

This notebook has been written to use the PySCF chemistry driver.

In [1]:
import numpy as np

from qiskit import BasicAer
from qiskit.transpiler import PassManager

from qiskit.aqua import Operator, QuantumInstance
from qiskit.aqua.algorithms.adaptive import VQE
from qiskit.aqua.algorithms.classical import ExactEigensolver
from qiskit.aqua.components.optimizers import L_BFGS_B
from qiskit.aqua.components.variational_forms import RYRZ

from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType

In [2]:
# using driver to get fermionic Hamiltonian
# PySCF example
driver = PySCFDriver(atom='H .0 .0 .0; H .0 .0 0.735', unit=UnitsType.ANGSTROM,
                     charge=0, spin=0, basis='sto3g')
molecule = driver.run()

In [3]:
# get fermionic operator and mapping to qubit operator
ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
qubitOp = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
qubitOp.to_matrix()
qubitOp.chop(10**-10)

In [4]:
# If you do have the driver installed or would like to start with a random Hamiltonian
# SIZE=4
# matrix = np.random.random((SIZE,SIZE))
# qubitOp = Operator(matrix=matrix)

In [5]:
# Using exact eigensolver to get the smallest eigenvalue
exact_eigensolver = ExactEigensolver(qubitOp, k=1)
ret = exact_eigensolver.run()
print('The exact ground state energy is: {}'.format(ret['eigvals'][0].real))

The exact ground state energy is: -1.8572750302023784


In [6]:
# setup VQE 
# setup optimizer, use L_BFGS_B optimizer for example
lbfgs = L_BFGS_B(maxfun=1000, factr=10, iprint=10)

# setup variational form generator (generate trial circuits for VQE)
var_form = RYRZ(qubitOp.num_qubits, 5, entangler_map = [[0, 1], [1, 2], [2, 3]])

# setup VQE with operator, variational form, and optimizer
vqe_algorithm = VQE(qubitOp, var_form, lbfgs, 'matrix')

backend = BasicAer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, pass_manager=PassManager())

results = vqe_algorithm.run(quantum_instance)
print("Minimum value: {}".format(results['eigvals'][0].real))
print("Parameters: {}".format(results['opt_params']))

Minimum value: -1.8532124263217393
Parameters: [-2.13953054  0.70800218 -0.17157494 -2.67458466  2.43244041  0.04126769
  0.34740155 -0.04775077 -1.151147    2.76097941 -1.48948796 -0.30086504
  0.7290411   2.40033569 -2.30581555  1.06377607 -2.97789243  1.43082718
 -0.91377262 -2.29316671 -0.04083006 -0.54650779 -2.43032826 -0.79940815
 -1.88176584  0.05495389  2.47406188 -0.82144629 -2.44818703 -3.11585379
 -2.54844951 -2.58470426 -0.99008597 -2.88926043  1.20856368  2.67069418
  2.4613227   1.22966774 -0.03176877  0.93517933  0.06694405  1.33700758
  1.49080935 -1.39533027  0.47972164  1.7949311  -3.01432916 -2.43192278]
